<a href="https://colab.research.google.com/github/m10923039/m10923039_hw01/blob/main/test01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import precision_recall_fscore_support
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Input, Dense, Activation,Dropout

讀取檔案

In [3]:
df_data=pd.read_csv('/content/drive/MyDrive/ObesityDataSet_raw_and_data_sinthetic.csv')

In [4]:
df_x=df_data.drop(['NObeyesdad'],axis=1)
df_y=df_data.loc[:,'NObeyesdad']
df_x['CH2O'].value_counts()
df_y.value_counts()

Obesity_Type_I         351
Obesity_Type_III       324
Obesity_Type_II        297
Overweight_Level_I     290
Overweight_Level_II    290
Normal_Weight          287
Insufficient_Weight    272
Name: NObeyesdad, dtype: int64

# **資料前處理**

## lable encoding

## family_history_with_overweight 超重的家族病史(是/否) <br>
no  -> 0 <br>
yes  -> 1
## SMOKE 抽煙(是/否) <br>
no  -> 0 <br>
yes  -> 1 <br>
## SCC 卡路里消耗監視(是/否)
no  -> 0 <br>
yes  -> 1 <br>
## CAEC 兩餐之間的食物消耗量
No  ->  1 <br>
no  ->  1 <br>
Sometimes  ->  2 <br>
Frequently  ->  3 <br>
Always  ->  4 <br>
## CALC 酒精消耗量
No  ->  1 <br>
no  ->  1 <br>
Sometimes  ->  2 <br>
Frequently  ->  3 <br>
Always  ->  4 <br>
# one hot 欄位
Gender 性別<br>
MTRANS 交通工具<br>

In [5]:
# df_x
df_x.loc()[df_x['family_history_with_overweight']=='no','family_history_with_overweight']=0
df_x.loc()[df_x['family_history_with_overweight']=='yes','family_history_with_overweight']=1

df_x.loc()[df_x['FAVC']=='no','FAVC']=0
df_x.loc()[df_x['FAVC']=='yes','FAVC']=1

df_x.loc()[df_x['SMOKE']=='no','SMOKE']=0
df_x.loc()[df_x['SMOKE']=='yes','SMOKE']=1

df_x.loc()[df_x['SCC']=='no','SCC']=0
df_x.loc()[df_x['SCC']=='yes','SCC']=1

df_x.loc()[df_x['CAEC']=='No','CAEC']=1
df_x.loc()[df_x['CAEC']=='no','CAEC']=1
df_x.loc()[df_x['CAEC']=='Sometimes','CAEC']=2
df_x.loc()[df_x['CAEC']=='Frequently','CAEC']=3
df_x.loc()[df_x['CAEC']=='Always','CAEC']=4

df_x.loc()[df_x['CALC']=='no','CALC']=1
df_x.loc()[df_x['CALC']=='Sometimes','CALC']=2
df_x.loc()[df_x['CALC']=='Frequently','CALC']=3
df_x.loc()[df_x['CALC']=='Always','CALC']=4

df_x[['family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC','CALC']] =df_x[['family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC','CALC']].astype(int)

x = pd.get_dummies(df_x)
y = pd.get_dummies(df_y)

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)
# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.15)

In [6]:
x_train.dtypes

Age                               float64
Height                            float64
Weight                            float64
family_history_with_overweight      int64
FAVC                                int64
FCVC                              float64
NCP                               float64
CAEC                                int64
SMOKE                               int64
CH2O                              float64
SCC                                 int64
FAF                               float64
TUE                               float64
CALC                                int64
Gender_Female                       uint8
Gender_Male                         uint8
MTRANS_Automobile                   uint8
MTRANS_Bike                         uint8
MTRANS_Motorbike                    uint8
MTRANS_Public_Transportation        uint8
MTRANS_Walking                      uint8
dtype: object

In [7]:
x_train

,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,Gender_Female,Gender_Male,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
1108,19.027417,1.588147,73.939268,1,0,3.000000,3.362758,2,0,2.000000,0,2.892922,1.000000,2,1,0,0,0,0,1,0
1079,24.751511,1.735343,83.337721,1,1,2.607335,3.000000,2,0,2.000000,0,0.451009,0.630866,2,0,1,0,0,0,1,0
2040,25.982261,1.629225,104.838425,1,1,3.000000,3.000000,2,0,2.556068,0,0.016820,0.582840,2,1,0,0,0,0,1,0
1871,21.001969,1.736215,132.145549,1,1,3.000000,3.000000,2,0,1.657541,0,1.672639,0.629285,2,1,0,0,0,0,1,0
556,18.000000,1.781543,50.869704,0,1,2.052932,3.000000,2,0,2.000000,0,0.520408,1.000000,2,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,22.142432,1.596110,42.848033,0,0,3.000000,2.581015,3,0,2.654702,0,0.902095,0.000000,1,1,0,0,0,0,1,0
1683,21.654800,1.755938,116.311962,1,1,1.800122,3.000000,2,0,2.000000,0,1.689525,0.764452,2,0,1,0,0,0,1,0
650,23.035829,1.598612,42.993937,0,0,2.753752,2.812377,3,0,2.346647,0,1.612248,0.000000,1,1,0,0,0,0,1,0
1233,22.661556,1.660324,94.189167,1,1,2.000000,3.000000,2,0,2.000000,0,0.000000,0.105936,1,0,1,0,0,0,1,0


In [8]:
y_train

,Insufficient_Weight,Normal_Weight,Obesity_Type_I,Obesity_Type_II,Obesity_Type_III,Overweight_Level_I,Overweight_Level_II
1108,0,0,0,0,0,0,1
1079,0,0,0,0,0,0,1
2040,0,0,0,0,1,0,0
1871,0,0,0,0,1,0,0
556,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...
514,1,0,0,0,0,0,0
1683,0,0,0,1,0,0,0
650,1,0,0,0,0,0,0
1233,0,0,1,0,0,0,0


In [9]:
input_layer = Input(shape=(x_train.shape[1],))
dense_layer_1 =Dense(32,kernel_initializer=tf.keras.initializers.GlorotNormal() ,activation='relu', name ="nl1" )(input_layer)
dense_layer_2 = Dense(16,kernel_initializer=tf.keras.initializers.GlorotNormal(),activation='relu', name ="nl2" )(dense_layer_1)
dense_layer_2 = Dense(10,activation='relu', name ="nl3" )(dense_layer_2)
dense_layer_2 = Dense(10,activation='relu', name ="nl4" )(dense_layer_2)
dense_layer_2 = Dropout(0.5)(dense_layer_2)
output = Dense(y_train.shape[1],activation='softmax')(dense_layer_2)
model = Model(inputs=input_layer, outputs=output)
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 21)]              0         
_________________________________________________________________
nl1 (Dense)                  (None, 32)                704       
_________________________________________________________________
nl2 (Dense)                  (None, 16)                528       
_________________________________________________________________
nl3 (Dense)                  (None, 10)                170       
_________________________________________________________________
nl4 (Dense)                  (None, 10)                110       
_________________________________________________________________
dropout (Dropout)            (None, 10)                0         
_________________________________________________________________
dense (Dense)                (None, 7)                 77    

In [10]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss',mode='min', patience=5)
Adam = tf.optimizers.Adam(lr=0.003, beta_1=0.9, beta_2=0.997, epsilon=1e-08, decay=0.0)
model.compile(optimizer=Adam, 
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics = [tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
              )
history = model.fit(x=x_train, y=y_train,  epochs=50,batch_size =3, callbacks=[callback] )

Epoch 1/50
563/563 [==============================] - 2s 1ms/step - loss: 2.2388 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 2/50
563/563 [==============================] - 1s 1ms/step - loss: 1.9296 - precision: 0.0054 - recall: 4.0314e-05
Epoch 3/50
563/563 [==============================] - 1s 1ms/step - loss: 1.8107 - precision: 0.4840 - recall: 0.0184
Epoch 4/50
563/563 [==============================] - 1s 1ms/step - loss: 1.6811 - precision: 0.4679 - recall: 0.0513
Epoch 5/50
563/563 [==============================] - 1s 1ms/step - loss: 1.6177 - precision: 0.6362 - recall: 0.0684
Epoch 6/50
563/563 [==============================] - 1s 1ms/step - loss: 1.6171 - precision: 0.6421 - recall: 0.0707
Epoch 7/50
563/563 [==============================] - 1s 1ms/step - loss: 1.5445 - precision: 0.6530 - recall: 0.0864
Epoch 8/50
563/563 [==============================] - 1s 1ms/step - loss: 1.5344 - precision: 0.6420 - recall: 0.0815
Epoch 9/50
563/563 [========================

**指標F1**

In [12]:
preds = model.evaluate(x = x_test, y = y_test)

F1 = 2 * (preds[1] * preds[2]) / (preds[1] + preds[2])
print('Precision', preds[1])
print('Recall}', preds[2])
print('F1',F1)


14/14 [==============================] - 0s 1ms/step - loss: 0.7201 - precision: 0.7529 - recall: 0.4610
Precision 0.7528957724571228
Recall} 0.4609929025173187
F1 0.5718475089082375


In [ ]:

# pd.DataFrame(history.history).plot(figsize = (8, 5))
# plt.grid(True)
# plt.gca().set_ylim(0, 2.5)
# plt.show()
